In [1]:
import joblib
import pickle
import numpy as np
import pandas as pd

# Load supervised model
xgb_pipeline = joblib.load("../models/xgb_pipeline.pkl")

# Load unsupervised model
iso_model = joblib.load("../models/isolation_forest_model.pkl")

# Load unsupervised preprocessing pipeline
iso_preprocess = joblib.load("../models/isolation_forest_preprocess.pkl")

# Load feature lists
with open("../feature_store/iso_features.pkl", "rb") as f:
    iso_features = pickle.load(f)


In [2]:
input_data = {
    "transaction_id": "T123",
    "account_id": "AC456",
    "customer_id": "C789",
    "customer_segment": "Gold",
    "transaction_datetime": "2025-10-02 15:35:00",
    "transaction_channel": "POS",
    "amount": 12000,
    "merchant_category": "Electronics",
    "transaction_location": "Lagos",
    "is_fraud": 0,  # ignored during inference
    "account_type": "Savings",

    # Engineered features (MUST be computed in real-time)
    "is_weekend": 0,
    "is_night": 0,
    "prev_txn_time": 3600,
    "time_since_last_transaction": 3600,
    "avg_amount_7d": 5200,
    "count_transactions_7d": 12,
    "hour": 15,
    "weekday": 3
}


In [3]:
df_new = pd.DataFrame([input_data])


In [4]:
supervised_prob = xgb_pipeline.predict_proba(df_new)[0][1]
supervised_pred = xgb_pipeline.predict(df_new)[0]

print("Supervised Probability of Fraud:", supervised_prob)
print("Supervised Prediction:", supervised_pred)


Supervised Probability of Fraud: 2.5478296e-06
Supervised Prediction: 0


In [5]:
# for the isolation forest
df_iso = df_new[iso_features]

processed_iso = iso_preprocess.transform(df_iso)

# -1 = anomaly, 1 = normal
anomaly_raw = iso_model.predict(processed_iso)[0]

# Convert to 1/0
anomaly_pred = 1 if anomaly_raw == -1 else 0

# Anomaly score: lower = more anomalous
anomaly_score = iso_model.decision_function(processed_iso)[0]
anomaly_score = -anomaly_score  # invert for "higher = more fraud-like"

print("Anomaly Prediction:", anomaly_pred)
print("Anomaly Score:", anomaly_score)


Anomaly Prediction: 0
Anomaly Score: -0.0509492398287108


In [6]:
fraud_risk = (
    0.7 * supervised_prob +    # XGBoost is primary
    0.3 * anomaly_score        # Isolation Forest assists
)

print("Final Fraud Risk Score:", fraud_risk)


Final Fraud Risk Score: -0.015282988467929214


In [7]:
threshold = 0.5   # configurable

final_decision = 1 if fraud_risk >= threshold else 0

print("Final Decision (1=Fraud):", final_decision)


Final Decision (1=Fraud): 0
